In [28]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler

In [29]:
os.getcwd()

'C:\\Users\\ashok.swarna\\OneDrive - Accenture\\cenlar'

In [30]:
def load_file():
    
    df = pd.read_excel('external_regressors.xlsx')
    df.head()
    df.reset_index(drop=True)
    df = df.set_index('Date')
    df.columns = df.columns.str.replace(' ', '_')
    cols = df.columns
    
    return df

In [51]:
def load_file_real():
    
    df = pd.read_excel('External_regressors_Auto.xlsx')
    df.head()
    df.reset_index(drop=True)
    df = df.set_index('Date')
    df.columns = df.columns.str.replace(' ', '_')
    cols = df.columns
    
    return df, cols

In [32]:
df_all = load_file()

In [33]:
df_all.head()

,Year,HPI,EMPL_non_farm_Employment,arm_adjustable_rates,population(Millions),30_year_Interest_rate_Mortgage,armrate,r-mortg,spread-mortg,1_year_Treasury,10_years_Treasury,treasury_Difference(Spread_treas),unemploymet_rate
Date,,,,,,,,,,,,,
2017-01-01,2017,184.770,145695,3.24,324.22,4.15,4.20,4.15,0.05,0.670000,2.211364,1.533944,4.7
2017-02-01,2017,185.138,145836,3.20,324.36,4.17,4.12,4.17,0.05,0.714286,2.298000,1.583714,4.7
2017-03-01,2017,186.642,145963,3.21,324.50,4.20,4.09,4.20,0.11,0.741935,2.481739,1.739804,4.4
2017-04-01,2017,188.649,146176,3.15,324.64,4.05,4.19,4.05,0.14,0.666667,2.182500,1.515833,4.4
2017-05-01,2017,190.652,146304,3.12,324.79,4.01,4.19,4.01,0.18,0.741935,2.203478,1.461543,4.4


In [52]:
df, col = load_file_real()

In [53]:
col

Index(['Forecast_key', 'Value_for_forecast_key'], dtype='object')

In [48]:
def fit_lstm(X_train, Y_train, X_test, Y_test ):
    #n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], Y_train.shape[1]
#fit LSTM
    #model = Sequential()
    #model.add(LSTM(300, activation='relu', input_shape=(X_train.shape[0], X_train.shape[1]),return_sequences = True))
    #model.add(Dropout(0.2))
    #model.add(LSTM(300))
    #model.add(Dropout(0.2))


    #model.compile(optimizer='adam', loss='mse')
# define model input_shape=(X_train.shape[0],X_train.shape[1]
    model = Sequential()
    model.add(LSTM(50, activation='tanh'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

#define model
 #   n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], Y_Train.shape[1]
# define model
    #model = Sequential()
    #model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
    #model.add(Dense(100, activation='relu'))
    #model.add(Dense(n_outputs))
    #model.compile(loss='mse', optimizer='adam')

# fitting the model
    model.fit(X_train, Y_train, epochs=10, batch_size=10)

    train_pred = model.predict(X_train, verbose=0)
    test_pred = model.predict(X_test, verbose=0)

    #train_pred.plot(marker="o", color='blue')
    #X_train.plot(color='black', marker="o", legend=True)
    #plt.show()
    #print(train_pred,test_pred )
    return model, train_pred, test_pred

In [37]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [38]:
def metrics(y_test, test_pred):
    
    MSE =mean_squared_error(y_true=y_test, y_pred=test_pred)                
    MAE = mean_absolute_error(y_true=y_test, y_pred=test_pred)
    MAPE = mean_absolute_percentage_error(y_test, test_pred)

    return MSE, MAE, MAPE

In [39]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [40]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    dff = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(dff.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(dff.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [54]:
#Extract the unique codes for filtering the over all data    
forecast_group = df.groupby('Forecast_key', as_index=False)
forecast_list = forecast_group.groups.keys()

In [56]:
df2 = pd.DataFrame()
for forecast_item in forecast_list:
    df1 = df[df['Forecast_key'].isin ([forecast_item])]
    df1=df1.sort_index()
    df3 = load_file()
    df3 = df3.astype('float32')
# normalize features
    #scaler = MinMaxScaler(feature_range=(0, 1))
    #scaled = scaler.fit_transform(df3)
# frame as supervised learning
   # reframed = series_to_supervised(scaled, 1, 1)
   # reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
   # print(reframed.head())
   # print(reframed.head())
    #df3.drop([forecast_item], axis =1, inplace = True)
    df4 = df3[forecast_item]
    
    X_train, X_test = df3[:27], df3[27:34]
    Y_train, Y_test = df4[:27], df4[27:34]
    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    Y_train = Y_train.to_numpy()
    Y_test = Y_test.to_numpy()
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
   # print(X_train.shape)
   # print(Y_train.shape)
    model, train_pred, test_pred = fit_lstm(X_train, Y_train, X_test, Y_test)

    train_mse, train_mae, train_mape = metrics(Y_train, train_pred)
    test_mse, test_mae, test_mape = metrics(Y_test, test_pred)
    print(forecast_item)
    print('train',train_mse, train_mae, train_mape )
    print('test',test_mse, test_mae, test_mape )
#    df3 = create_data_frame(df1, train, test, linear_pred_tr,tr_linear_mape, expo_pred_tr,tr_expo_mape,add_pred_tr, tr_add_mape ,linear_pred_te,te_linear_mape,expo_pred_te,te_expo_mape,add_pred_te,te_add_mape)
#    df2 = pd.concat([df2,df3], ignore_index = True)

#df2

Epoch 1/10
27/27 [==============================] - 13s 486ms/step - loss: 7.7160
Epoch 2/10
27/27 [==============================] - 0s 798us/step - loss: 7.6241
Epoch 3/10
27/27 [==============================] - 0s 745us/step - loss: 7.5327
Epoch 4/10
27/27 [==============================] - 0s 862us/step - loss: 7.4409
Epoch 5/10
27/27 [==============================] - 0s 1ms/step - loss: 7.3520
Epoch 6/10
27/27 [==============================] - 0s 775us/step - loss: 7.2618
Epoch 7/10
27/27 [==============================] - 0s 679us/step - loss: 7.1728
Epoch 8/10
27/27 [==============================] - 0s 697us/step - loss: 7.0854
Epoch 9/10
27/27 [==============================] - 0s 656us/step - loss: 6.9972
Epoch 10/10
27/27 [==============================] - 0s 798us/step - loss: 6.9117
10_years_Treasury
train 6.850643 2.6018918 103.6816954612732
test 4.249764 2.0383043 104.80504035949707
Epoch 1/10
27/27 [==============================] - 13s 486ms/step - loss: 0.6322
Epoc

Epoch 9/10
27/27 [==============================] - 0s 554us/step - loss: 17.1214
Epoch 10/10
27/27 [==============================] - 0s 554us/step - loss: 16.8914
r-mortg
train 16.732964 4.0790296 95.33835649490356
test 13.113159 3.6159081 94.78576183319092
Epoch 1/10
27/27 [==============================] - 7s 249ms/step - loss: 15.4808
Epoch 2/10
27/27 [==============================] - 0s 554us/step - loss: 15.1368
Epoch 3/10
27/27 [==============================] - 0s 739us/step - loss: 14.7970
Epoch 4/10
27/27 [==============================] - 0s 480us/step - loss: 14.4602
Epoch 5/10
27/27 [==============================] - 0s 480us/step - loss: 14.1293
Epoch 6/10
27/27 [==============================] - 0s 443us/step - loss: 13.8042
Epoch 7/10
27/27 [==============================] - 0s 517us/step - loss: 13.4808
Epoch 8/10
27/27 [==============================] - 0s 554us/step - loss: 13.1652
Epoch 9/10
27/27 [==============================] - 0s 517us/step - loss: 12.8530
Ep